In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Load Data
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 2. Filter: from LOYAL customers
loyal_customers = df[df['Churn'] == 'No'].copy()

# 3. Select only "Service" columns
services = ['PhoneService', 'MultipleLines', 'InternetService',
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
            'TechSupport', 'StreamingTV', 'StreamingMovies']

basket = loyal_customers[services]

# 4. Convert text to Boolean (True/False)
def encode_units(x):
    if x in ['No', 'No internet service', 'No phone service']:
        return False
    else:
        return True

basket = basket.map(encode_units)

# 5. Run Apriori
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# 6. Generate Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 7. FILTERING
def is_interesting(consequent):
    # Convert frozenset to list
    items = list(consequent)
    if items == ['InternetService'] or items == ['PhoneService'] or items == ['MultipleLines']:
        return False
    return True

# Apply the filter
rules['is_interesting'] = rules['consequents'].apply(is_interesting)
interesting_rules = rules[rules['is_interesting'] == True].sort_values(['lift'], ascending=False)

# 8. Display the "Gold" Rules
print("--- TOP 5 UPSELLING OPPORTUNITIES (Based on Loyal Customers) ---")
cols_to_show = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
display(interesting_rules[cols_to_show].head(5))

# Final output
print("\n--- AUTOMATED INSIGHT ---")
top_rule = interesting_rules.iloc[0]
ant = list(top_rule['antecedents'])[0]
cons = list(top_rule['consequents'])[0]
print(f"Strongest Pattern Found: Customers who have {ant} are very likely to buy {cons}.")
print(f"Strategy: Filter 'At Risk' customers who have {ant} but NOT {cons}, and offer them a discount on {cons}.")

--- TOP 5 UPSELLING OPPORTUNITIES (Based on Loyal Customers) ---


,antecedents,consequents,support,confidence,lift
12760,"(MultipleLines, TechSupport, StreamingTV)","(DeviceProtection, OnlineBackup, PhoneService,...",0.056243,0.484193,3.695007
13990,"(MultipleLines, TechSupport, StreamingTV)","(PhoneService, OnlineBackup, InternetService, ...",0.056243,0.484193,3.695007
13871,"(PhoneService, OnlineBackup, InternetService, ...","(MultipleLines, TechSupport, StreamingTV)",0.056243,0.429204,3.695007
13925,"(DeviceProtection, OnlineBackup, PhoneService,...","(InternetService, MultipleLines, TechSupport, ...",0.056243,0.429204,3.695007
13936,"(InternetService, MultipleLines, TechSupport, ...","(DeviceProtection, OnlineBackup, PhoneService,...",0.056243,0.484193,3.695007



--- AUTOMATED INSIGHT ---
Strongest Pattern Found: Customers who have MultipleLines are very likely to buy DeviceProtection.
Strategy: Filter 'At Risk' customers who have MultipleLines but NOT DeviceProtection, and offer them a discount on DeviceProtection.
